In [76]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset

In [77]:
training_data = pd.read_csv("data/train.csv")
training_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [78]:
training_data.fillna(0, inplace=True)
training_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [79]:
training_data = training_data.to_numpy()
training_data

array([['0001_01', 'Europa', False, ..., 0.0, 'Maham Ofracculy', False],
       ['0002_01', 'Earth', False, ..., 44.0, 'Juanna Vines', True],
       ['0003_01', 'Europa', False, ..., 49.0, 'Altark Susent', False],
       ...,
       ['9279_01', 'Earth', False, ..., 0.0, 'Fayey Connon', True],
       ['9280_01', 'Europa', False, ..., 3235.0, 'Celeon Hontichre',
        False],
       ['9280_02', 'Europa', False, ..., 12.0, 'Propsh Hontichre', True]],
      dtype=object)

In [80]:
training_data[:, 3][0].split('/')

['B', '0', 'P']

In [81]:
def split_3rd(x):
    if x == 0:
        return [0, 0, 0]
    else:
        a, b, c = x.split('/')
        return a, int(b), c


new_seperated_column = np.hstack(map(split_3rd, training_data[:, 3])).reshape(training_data.shape[0], 3)

C:\Users\shaow\AppData\Local\Temp\ipykernel_39768\1914284128.py:9: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  new_seperated_column = np.hstack(map(split_3rd, training_data[:, 3])).reshape(training_data.shape[0], 3)


In [82]:
training_data = np.delete(training_data, 3, axis=1)

In [83]:
training_data = np.hstack((training_data, new_seperated_column))
training_data

array([['0001_01', 'Europa', False, ..., 'B', '0', 'P'],
       ['0002_01', 'Earth', False, ..., 'F', '0', 'S'],
       ['0003_01', 'Europa', False, ..., 'A', '0', 'S'],
       ...,
       ['9279_01', 'Earth', False, ..., 'G', '1500', 'S'],
       ['9280_01', 'Europa', False, ..., 'E', '608', 'S'],
       ['9280_02', 'Europa', False, ..., 'E', '608', 'S']], dtype=object)

In [84]:
def class_mapping(column_number):
    classes = {}
    index = 0
    for id in training_data[:, column_number]:
        if id not in classes:
            classes[id] = index
            index += 1
    return classes

In [85]:
map_planet = class_mapping(1)

In [86]:
training_data[:, 1] = np.vectorize(map_planet.get)(training_data[:, 1])
training_data

array([['0001_01', 0, False, ..., 'B', '0', 'P'],
       ['0002_01', 1, False, ..., 'F', '0', 'S'],
       ['0003_01', 0, False, ..., 'A', '0', 'S'],
       ...,
       ['9279_01', 1, False, ..., 'G', '1500', 'S'],
       ['9280_01', 0, False, ..., 'E', '608', 'S'],
       ['9280_02', 0, False, ..., 'E', '608', 'S']], dtype=object)

In [87]:
def replace_with_mapped_value(column_number, map_name, data_set):
    data_set[:, column_number] = np.vectorize(map_name.get)(data_set[:, column_number])

In [88]:
TF_map = class_mapping(2)
TF_map

{False: 0, True: 1}

In [89]:
destination_map = class_mapping(3)
destination_map

{'TRAPPIST-1e': 0, 'PSO J318.5-22': 1, '55 Cancri e': 2, 0: 3}

In [90]:
cabin0 = class_mapping(13)
cabin0

{'B': 0, 'F': 1, 'A': 2, 'G': 3, '0': 4, 'E': 5, 'D': 6, 'C': 7, 'T': 8}

In [91]:
cabin2 = class_mapping(15)
cabin2

{'P': 0, 'S': 1, '0': 2}

In [92]:
replace_with_mapped_value(2, TF_map, training_data)
replace_with_mapped_value(3, destination_map, training_data)
replace_with_mapped_value(5, TF_map, training_data)
replace_with_mapped_value(13, cabin0, training_data)
replace_with_mapped_value(15, cabin2, training_data)

In [93]:
training_data

array([['0001_01', 0, 0, ..., 0, '0', 0],
       ['0002_01', 1, 0, ..., 1, '0', 1],
       ['0003_01', 0, 0, ..., 2, '0', 1],
       ...,
       ['9279_01', 1, 0, ..., 3, '1500', 1],
       ['9280_01', 0, 0, ..., 5, '608', 1],
       ['9280_02', 0, 0, ..., 5, '608', 1]], dtype=object)

In [94]:
np.random.shuffle(training_data)
training_data, test_data = training_data[:int(len(training_data) * 0.8)], training_data[int(len(training_data) * 0.8):]


# x=np.delete(training_data,[0,1,2],1).astype(float)
# y=training_data[:,[1]].astype(float)
# tensor_x=torch.Tensor(x)
# tensor_y=torch.Tensor(y)
# training_data=TensorDataset(tensor_x,tensor_y)
def to_tensor(np_array, removed_columns, target_column):
    removed_columns.append(target_column)
    x = np.delete(np_array, removed_columns, axis=1).astype(float)
    y = np_array[:, target_column].astype(int)
    y_with_zeros = []
    for class_element in y:
        y_with_zeros.append(np.zeros(len(TF_map)))
        y_with_zeros[-1][TF_map[class_element]] = 1
    y = y_with_zeros
    tensor_x = torch.Tensor(x)
    tensor_y = torch.Tensor(y)
    tensor = TensorDataset(tensor_x, tensor_y)
    return tensor

In [95]:
batch_size = 128

# Create data loaders.
train_dataloader = DataLoader(to_tensor(training_data, [0, 11], 12), batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(to_tensor(test_data, [0, 11], 12), batch_size=batch_size)

for X, y in train_dataloader:
    print(X.shape)
    print(y.shape, y.dtype)
    break

# # Display sample data
# figure = plt.figure(figsize=(10, 8))
# cols, rows = 5, 5
# for i in range(1, cols * rows + 1):
#     idx = torch.randint(len(test_data), size=(1,)).item()
#     img, label = test_data[idx]
#     figure.add_subplot(rows, cols, i)
#     plt.title(label)
#     plt.axis("off")
#     plt.imshow(img.squeeze(), cmap="gray")
# plt.show()

torch.Size([128, 13])
torch.Size([128, 2]) torch.float32


In [96]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))
layer_size = 512
input_size = 13


# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.Sigmoid(),
            nn.Linear(layer_size, layer_size),
            nn.Sigmoid(),
            nn.Linear(layer_size, layer_size),
            nn.Sigmoid(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.ReLU(),
            nn.Linear(layer_size, layer_size),
            nn.Sigmoid(),
            nn.Linear(layer_size, layer_size),
            nn.Sigmoid(),
            nn.Linear(layer_size, len(TF_map)),
            nn.Sigmoid(),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=13, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=512, bias=True)
    (5): ReLU()
    (6): Linear(in_features=512, out_features=512, bias=True)
    (7): Sigmoid()
    (8): Linear(in_features=512, out_features=512, bias=True)
    (9): Sigmoid()
    (10): Linear(in_features=512, out_features=512, bias=True)
    (11): Sigmoid()
    (12): Linear(in_features=512, out_features=512, bias=True)
    (13): ReLU()
    (14): Linear(in_features=512, out_features=512, bias=True)
    (15): ReLU()
    (16): Linear(in_features=512, out_features=512, bias=True)
    (17): ReLU()
    (18): Linear(in_features=512, out_features=512, bias=True)
    (19): Sigmoid()
    (20): Linear(in_features=512, out_features=512, bias=True)
    (21): Sigmoid()
    (22): Linear(in_features=512, out_features=2

In [97]:
# loss_fn = nn.L1Loss()
loss_fn = nn.MSELoss()
learning_rate = 1e-5
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [98]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # y=y.type(torch.float)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 8 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


In [99]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # print(pred.argmax(1).shape,y.shape)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    try:
        if correct>0.81:
            torch.save(model,'model/m3')
        elif correct>0.805:
            torch.save(model,'model/m2')
        elif correct>0.80:
            torch.save(model,'model/m1')
            print('save success')
    except Exception:
        pass


    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [100]:
epochs = 1000
for t in range(epochs):
    print(f"Epoch {t + 1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 0.256395  [    0/ 6954]
loss: 0.268650  [ 1024/ 6954]
loss: 0.255308  [ 2048/ 6954]
loss: 0.247280  [ 3072/ 6954]
loss: 0.244419  [ 4096/ 6954]
loss: 0.268394  [ 5120/ 6954]
loss: 0.249724  [ 6144/ 6954]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.002018 

Epoch 2
-------------------------------
loss: 0.250016  [    0/ 6954]
loss: 0.249838  [ 1024/ 6954]
loss: 0.252416  [ 2048/ 6954]
loss: 0.251577  [ 3072/ 6954]
loss: 0.248714  [ 4096/ 6954]
loss: 0.251386  [ 5120/ 6954]
loss: 0.249815  [ 6144/ 6954]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.002012 

Epoch 3
-------------------------------
loss: 0.250037  [    0/ 6954]
loss: 0.249820  [ 1024/ 6954]
loss: 0.250218  [ 2048/ 6954]
loss: 0.250369  [ 3072/ 6954]
loss: 0.249665  [ 4096/ 6954]
loss: 0.249621  [ 5120/ 6954]
loss: 0.250367  [ 6144/ 6954]
Test Error: 
 Accuracy: 50.5%, Avg loss: 0.002012 

Epoch 4
-------------------------------
loss: 0.249821  [    0/ 6954]
loss: 0.250141  [ 1024/ 


KeyboardInterrupt



In [ ]:
torch.save(model, 'model/m0')

In [ ]:
final_test_data = pd.read_csv("data/test.csv")
final_test_data.fillna(0, inplace=True)
final_test_data = final_test_data.to_numpy()
new_seperated_column = np.hstack(map(split_3rd, final_test_data[:, 3])).reshape(final_test_data.shape[0], 3)
final_test_data = np.delete(final_test_data, 3, axis=1)
final_test_data = np.hstack((final_test_data, new_seperated_column))
final_test_data

In [ ]:
replace_with_mapped_value(1, map_planet, final_test_data)
replace_with_mapped_value(2, TF_map, final_test_data)
replace_with_mapped_value(3, destination_map, final_test_data)
replace_with_mapped_value(5, TF_map, final_test_data)
replace_with_mapped_value(12, cabin0, final_test_data)
replace_with_mapped_value(14, cabin2, final_test_data)
final_test_data = np.delete(final_test_data, 11, axis=1)
final_test_data

In [ ]:
final_test_data[:, 1:]

In [ ]:
model = torch.load('model/m2')

In [ ]:
dct = {v: k for k, v in TF_map.items()}
dct

In [ ]:
result = []
for line in final_test_data:
    current_line = torch.Tensor(line[1:].astype(float)).to(device)
    pred = model(current_line)
    result.append([line[0], dct[pred.argmax().item()]])
result

In [ ]:
result = [['PassengerId', 'Transported']] + result
result

In [ ]:
np.savetxt("result.csv", result, delimiter=",", fmt='%s')